# Pedersen Study

Analyze the performance of the model against the extreme downward refraction
N^2 linear test case developed by Pedersen and Gordon.  This is also the
test case that Weinberg and Keenan used to calibrate the minimum beam width
in GRAB. This test serves several important functions
- Compares the ray path locations, travel time, and D/E angles to analytic results.
- Compares the total propagation loss to analytic results.
- Analyzes the contribution of the phase change at the caustic.

The Pedersen/Gordon paper focuses two test cases.

- The shallow source portion analyzed an acoustic field for a downward
  refracting profile that had been fit to experimental velocity data.
  Propagation loss was studied in both the image interference zone
  and the shadow zone.
- The deep source test cases extended this analysis to a non-physical
  extreme that emphasizes the effects of the caustic.

Pedersen/Gordon concludes that the pi/2 phase change should only occur
when the ray path touches the caustic and not when the ray passes through
a horizontal vertex.  This is consistent with the current implementation
of GRAB, but different than the original Weinberg/Keenan paper.

Uses the MKS version of the parameters defined in Jensen et. al.
instead of the original yd/sec version defined by Pedersen.

- c0 = 1550 m/s, g0 = 1.2 1/s, f = 2000 Hz
- shallow:
    - source: depth = 75 m, time inc=0.025, cs = 1467.149925 m/s
    - target: depth = 75 m, range = 500-1000 m
- deep:
    - source: depth = 1000 m, time inc=0.1, cs = 970.954770 m/s
    - target: depth = 800 m, range = 3000-3100 m
 
 ---
 **Refefences:**
 * M. A. Pedersen, D. F. Gordon, "Normal-Mode and Ray Theory Applied
   to Underwater Acoustic conditions of Extreme Downward Refraction",
   J. Acoust. Soc. Am. 51 (1B), 323-368 (June 1972).
 * H. Weinberg, R. E. Keenan, “Gaussian ray bundles for modeling
   high-frequency propagation loss under shallow-water conditions”,
   J. Acoust. Soc. Am. 100 (3), 1421-1431, (Sept 1996).
 * L. M. Brekhovskikh, Waves in Layered Media, 2nd Edition,
   Academic Press Inc., 1980, Section 54.
 * F. B. Jensen, W. A. Kuperman, M. B. Porter, and H. Schmidt,
   Computational Ocean Acoustics (American Institute of Physics Press,
   New York, 1994) pp. 162-166.

# Setup path information

Setup USML path information assuming that this notebook is in usml/python/test directory.

In [1]:
import sys
import os

USML_DIR = os.path.abspath("../..")
sys.path.append(os.path.join(USML_DIR,"python/src"))

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import usml.netcdf
import usml.plot

# Shallow Source Raytrace

Illustrates the effects of downward refraction and surface reflection wavefront shapes. The colored lines below illustrate ray paths with launch angles from zero to 25 degrees upward, at a spacing of 1.0 degrees. Similar to Figure 3.6 in Jensen et. al. The critical ray for a source at 75.0 meters is 18.82 degrees and launch angles below this refract down before hitting the ocean surface. The black curve represents the acoustic wavefront shape for the travel time specified in the slider bar. The wavefront points are sampled at a 0.02 degrees and this causes the wavefront to stretch into the shadow zone slightly farther than the ray drawing. The surface reflection creates a fold in the wavefront. This creates a shadow zone near the ocean surface, but it does not create a caustic.

In [4]:
filename = os.path.join(USML_DIR, "studies/pedersen/pedersen_shallow_raytrace.nc")
wavefront = usml.netcdf.read(filename)
de_list = wavefront.source_de[::50]
de_direct = de_list[ np.asarray(de_list < 18.82).nonzero() ]
de_reflect = de_list[ np.asarray(de_list >= 18.82).nonzero() ]

tmin = wavefront.travel_time[0]
tmax = wavefront.travel_time[-1]
tinc = wavefront.travel_time[1] - wavefront.travel_time[0]

@interact
def shallow_raytrace(time=(tmin,tmax,tinc)):
    fig, ax = plt.subplots(figsize=(8, 6))
    usml.plot.plot_raytrace_2d(ax,wavefront,de=de_direct)
    usml.plot.plot_raytrace_2d(ax,wavefront,de=de_reflect,fmt="--")
    usml.plot.plot_wavefront_2d(ax,wavefront,time=time,fmt="k-")
    ax.grid(True)
    ax.set_xlabel('Range (km)')
    ax.set_ylabel('Depth (m)')
    ax.set_xlim(0,1.2)
    ax.set_ylim(-500,0)

echo = 0

interactive(children=(FloatSlider(value=0.42, description='time', max=0.8500000000000005, step=0.01), Output()…

# Deep Source Raytrace

Illustrates the effect of caustic on wavefront folds. The colored lines below illustrate ray paths with launch angles from 20 to 60 degrees upward, at a spacing of 10 degrees. The critical ray for a source at 1000.0 meters is 51.21 degrees and launch angles below this refract down before hitting the ocean surface. The black curve represents the acoustic wavefront shape for the travel time specified in the slider bar. The wavefront points are sampled at a 0.2 degrees and this causes the wavefront to stretch into the shadow zone slightly farther than the ray drawing. The surface reflection creates a fold in the wavefront. This creates a shadow zone near the ocean surface. At times greater than about 2.5 seconds, there is a caustic in the direct path.  So points on trailing side of the wavefront fold can have different numbers of surface reflections.

In [3]:
filename = os.path.join(USML_DIR, "studies/pedersen/pedersen_deep_raytrace.nc")
wavefront = usml.netcdf.read(filename)

de_list = wavefront.source_de[::10]
de_direct = de_list[ np.asarray(de_list < 51.21).nonzero() ]
de_reflect = de_list[ np.asarray(de_list >= 51.21).nonzero() ]

tmin = wavefront.travel_time[0]
tmax = wavefront.travel_time[-1]
tinc = wavefront.travel_time[1] - wavefront.travel_time[0]

@interact
def deep_raytrace(time=(tmin,tmax,tinc)):
    fig, ax = plt.subplots(figsize=(8, 6))
    usml.plot.plot_raytrace_2d(ax,wavefront,de=de_direct)
    usml.plot.plot_raytrace_2d(ax,wavefront,de=de_reflect,fmt="--")
    usml.plot.plot_wavefront_2d(ax,wavefront,time=time,fmt="k-")
    ax.grid(True)
    ax.set_xlabel('Range (km)')
    ax.set_ylabel('Depth (m)')
    ax.set_xlim(0,3.5)
    ax.set_ylim(-1200,0)

echo = 0

interactive(children=(FloatSlider(value=1.75, description='time', max=3.509999999999969, step=0.01), Output())…

# Analytic Model for Ray Paths in $n^2$ Linear Environment

Analytic solution for the ray paths of the Pedersen profile in a Spherical coordinate system. The Pedersen profile is an extreme test case for a downward refracting $n^2$ linear profile.

$$
c(r) = \left[ \frac{c_0}{ \sqrt{ \frac{ 1 + 2 g_0 \left( R - r \right) }{c_0} } } \right] \frac{r}{R}
$$

where:
* $R$ = Earth radius of curvature
* $r$ = radial component of position (positive is up)
* $c_0$ = sound speed at the ocean surface
* $g_0$ = sound speed gradient at at the ocean surface

A "flat Earth" correction has been applied to the speed profile to simplify comparisons to the Cartesian coordinate system.

This test produces an analytic solution for the cycle range as a function of launch angle.  For the purpose of this test, the cycle range is
defined as the range required to reach the upward refracting apex and return to the original depth. For a source at 1000 yds, only angles in
the range (0,50.44] degrees (up) will produce a purely refractive cycle range.  Following Pedersen example, we will focus our analysis on angles
in the range [20,50].

The range from the source to the first vertex is computed by integrating the equation for dQ/dr using Snell's law for spherical media and the
quadgk() function.

$$
r \frac{dQ}{dr} = cot(A) = \frac{ac}{\sqrt{1 - (ac)^2} }
$$

$$
a = \frac{r cos(A_1)}{c_1}
$$

where:

* $Q$ = latitude component of position (positive is south)
* $A$ = depression/elevation angle at each r (positive is up)
* $A_1$ = launch angle (positive is up)
* $c_1$ = sound speed at source depth
* $a$ = ray parameter (constant for each launch angle)

To complete the computation of cycle range, distances from upward and
downward launch angles are combined to compute the complete distance.
Note that the fact that we are working in Spherical media adds extra
factors of "r" to both dQ/dr and Snell's law.

# Shallow Source Analytics Solution

# Shallow Source Propagation Loss

Tests the accuracy of the eigenray_collection model against the analytic solution for the shallow source, N^2 linear test case developed by Pedersen and Gordon. The source is located at a depth of 75 meters. Receivers have a depth of 75 meters and ranges from 500 to 1000 yds. Uses rays from 0 to 30 degrees at a very a tight 0.025 deg spacing.  Such tight spacing is needed to capture the portion of the surface reflected path just inside of the critical ray. Tests the model's ability to produce accurate propagation loss, phase, travel times, and eigneray angles at the edge of a shadow zone. 

# Deep Source Propagation Loss

Tests the accuracy of the eigenray_collection model against the analytic solution for the deep source, N^2 linear test case developed by Pedersen and Gordon. The source is located at a depth of 1000 meters. Receivers have a depth of 800 meters and ranges from 3000 to 3120 meters. Uses rays from 20 to 60 degrees at a 0.25 deg spacing. Tests the model's ability to produce accurate propagation loss, phase, travel times, and eigneray angles in an area where a caustic causes rays to cross at the edge of a shadow zone.

We found that tangent spaced beams did not work very well for this scenario.  Several combinations of parameters lead to artifacts in the
shadow zone from an imperfect cancellation of the direct and caustic paths. We believe that this is caused the accumulation of small cell width
and target range errors over hundreds of contributing beams.